In [175]:
import pandas as pd
from pymongo import MongoClient
import folium

In [176]:
locales = pd.read_csv("../output/requisitos.csv")
display(locales.head(),locales.shape)

,Unnamed: 0,category,name,location,latitude,longitude
0,0,airport,San Francisco International Airport,"{'type': 'Point', 'coordinates': [-122.3789554...",37.621313,-122.378955
1,0,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.406831,...",37.789348,-122.406831
2,1,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.4042684...",37.792814,-122.404268
3,2,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.38783, ...",37.741198,-122.387830
4,3,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.431519,...",37.782738,-122.431519


(106, 6)

In [177]:
locales.category.value_counts()

Starbucks     59
pub           40
vegan rest     6
airport        1
Name: category, dtype: int64

In [199]:
# coord SF [37.774929,-122.4194183]
m = folium.Map(
    location=[37.774929,-122.4194183],
    zoom_start=13,
    tiles='OpenStreetMap'
)
for i in range(len(locales.index)):
    dict_color ={"airport":"black","Starbucks":"blue","vegan rest":"green","pub":"red"}
    folium.Marker(
    location=[float(locales.iloc[i][4]),float(locales.iloc[i][5])],
    popup=locales.iloc[i][1],
    icon=folium.Icon(color=dict_color[locales.iloc[i][1]])).add_to(m)

folium.Circle(
    radius=1000,
    location=[37.7695, -122.4130],
    color='black',
    fill=False,
).add_to(m)


m

In [200]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [208]:
#compañias en el cículo [37.7695, -122.4130]
cur = db.companiesSF.find({"office": {
         "$near": {
           "$geometry": {"type" : "Point" ,"coordinates":[-122.4130, 37.7695]},
           "$maxDistance": 1000}}})

In [209]:
df = pd.DataFrame(cur)
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,city,office,state,total_money_raised ($k),money_state
0,5e9d3c7155122cdd5bd9ccd7,Kontera,advertising,NaN,2003.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.411157,...",success,36100.0,success
1,5e9d3c7155122cdd5bd9cd85,PopularMedia,advertising,NaN,2003.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.411157,...",success,12300.0,success
2,5e9d3c7155122cdd5bd9d00c,UTOPY,software,40.0,1999.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.411157,...",success,7500.0,success
3,5e9d3c7155122cdd5bd9cdf9,Infectious,web,4.0,2007.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.410217,...",success,750.0,success
4,5e9d3c7155122cdd5bd9cedb,Plum,web,NaN,NaN,San Francisco,"{'type': 'Point', 'coordinates': [-122.414952,...",success,0.0,success2


(133, 10)

In [210]:
#locales en el cículo [-122.4130, 37.7695]
cur = db.locales.find({"location": {
         "$near": {
           "$geometry": {"type" : "Point" ,"coordinates":[-122.4130, 37.7695]},
           "$maxDistance": 1000}}})

In [211]:
df_locales_circulo=pd.DataFrame(cur)

In [212]:
df_locales_circulo.head()

,_id,category,name,location,latitude,longitude
0,5e9d7a8255122cdd5bd9de55,pub,DNA Lounge,"{'type': 'Point', 'coordinates': [-122.4126502...",37.771094,-122.412650
1,5e9d7a8255122cdd5bd9de37,pub,Audio Nightclub,"{'type': 'Point', 'coordinates': [-122.413792,...",37.771362,-122.413792
2,5e9d7a8255122cdd5bd9de5e,pub,The Holy Cow,"{'type': 'Point', 'coordinates': [-122.4143364...",37.771373,-122.414336
3,5e9d7a8255122cdd5bd9de39,pub,Calle-11sf Nightclub,"{'type': 'Point', 'coordinates': [-122.414007,...",37.771587,-122.414007
4,5e9d7a8255122cdd5bd9de3f,pub,OASIS,"{'type': 'Point', 'coordinates': [-122.4144444...",37.771944,-122.414444


In [214]:
# coord cículo [-122.4130, 37.7695]
m = folium.Map(
    location=[37.7695,-122.4130],
    zoom_start=14,
    tiles='OpenStreetMap'
)
for i in range(len(df_locales_circulo.index)):
    dict_color ={"Starbucks":"blue","vegan rest":"green","pub":"red"}
    folium.Marker(
    location=[float(df_locales_circulo.iloc[i][4]),float(df_locales_circulo.iloc[i][5])],
    popup=df_locales_circulo.iloc[i][1],
    icon=folium.Icon(color=dict_color[df_locales_circulo.iloc[i][1]])).add_to(m)

for i in range(len(df.index)):
    folium.Marker(
    location=[float(df.iloc[i][6]["coordinates"][1]),float(df.iloc[i][6]["coordinates"][0])],
    popup=df.iloc[i][1],
    icon=folium.Icon(color="black")).add_to(m)

folium.Circle(
    radius=1000,
    location=[37.7695,-122.4130],
    color='black',
    fill=False,
).add_to(m)


m